In [1]:
import sys
import statistics

from qetu_sim import *

import numpy as np
import scipy.linalg
import itertools
import matplotlib.pyplot as plt

from qiskit import *
from qiskit.transpiler import Layout
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import Permutation
from qiskit.visualization import *
from qiskit_aer import AerSimulator

In [2]:
u = 1
t = 1
delta_t = 1
n = 1
num_sites = 4

In [3]:
H_ref = ref_fh_hamiltonian(u=u, t=t, WMI_qubit_layout=True, include_aux=True)
λ, v = np.linalg.eigh(H_ref)
ground_state_energy = λ[0]
ground_state_vector = v[:,0]

In [4]:
from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
pauli_error, depolarizing_error, thermal_relaxation_error)
error_1q = 1e-3
error_2q = 1e-2
error_readout = 1e-1

# QuantumError objects
depol_error_1 = depolarizing_error(error_1q, 1)
depol_error_2 = depolarizing_error(error_2q, 2)

noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(depol_error_1, ["sx", "sy", "x", "y"])
noise_model.add_all_qubit_quantum_error(depol_error_2, ["cp", "pswap"])

noise_model.add_basis_gates(['unitary'])


In [5]:
#noise_model = define_noise_model(t1=200e3, t2=300e3, duration_1q=20, duration_2q=60, duration_readout=1500, fidelity_1q=0.999, fidelity_2q=0.995, fidelity_readout=0.98)

sim_ideal = AerSimulator()
sim_noise = AerSimulator(noise_model=noise_model)

simulator = sim_noise

degree_list = range(2,51,2)
trotter_steps_list = range(1,2)
energy_estimation_list = []
overlap_list = []
statevector_list = []

for degree in degree_list:
    E_min, E_mu_m, E_mu_p, E_max = calculate_qsp_params(u, t)
    qsp = QSPPhase()
    phi_seq_su2 = qsp.cvx_qsp_heaviside(
        degree,
        E_min,
        E_mu_m, 
        E_mu_p, 
        E_max
    )
    phi_vec = convert_Zrot_to_Xrot(phi_seq_su2)
    for trotter_steps in trotter_steps_list:
        print("Degree: " + str(degree) + "\t Trotter steps: " + str(trotter_steps))
        # Construct quantum circuit
        QETU_circ = construct_QETU_circ(u, t, trotter_steps, phi_vec)
        QETU_circ_WMI = transpile_QETU_to_WMI(QETU_circ)
        QETU_circ_WMI = add_pswap_labels(QETU_circ_WMI)
        QETU_circ_WMI = add_sy_labels(QETU_circ_WMI)
        QETU_circ_WMI = transpile(QETU_circ_WMI, simulator, basis_gates=['pswap', 'cp', 'rz', 'sx', 'sy', 'x', 'y', 'unitary'])
        # prepare ground state
        initial_state = Statevector.from_label("+01+0-10-")
        circuit = QuantumCircuit(9)
        circuit.initialize(initial_state)
        circuit.compose(QETU_circ_WMI, inplace=True)
        circuit.save_statevector(pershot=True)
        result = simulator.run(transpile(circuit, simulator, basis_gates=['pswap', 'cp', 'rz', 'sx', 'sy', 'x', 'y', 'unitary', 'save_statevector']), shots=1000).result()
        overlap_shots = []
        energy_shots = []
        final_state_list = []
        for psi in result.data(0)['statevector']:
            final_state = psi.data.real
            final_state_norm = final_state / np.linalg.norm(final_state, 2)
            final_state_list.append(final_state_norm)
            overlap = abs(np.vdot(final_state_norm, ground_state_vector))**2
            overlap_shots.append(overlap)
        # estimate ground state
        E0_meas = estimate_ground_state_energy_from_statevector_list(final_state_list, u=1, t=1, use_num_conservation=True)
        energy_shots.append(E0_meas)
        overlap_list.append(overlap_shots)
        statevector_list.append(final_state_list)
        energy_estimation_list.append(energy_shots)
        print("mean overlap: " + str(statistics.mean(overlap_shots)))
        print("min overlap: " + str(min(overlap_shots)))
        print("max overlap: " + str(max(overlap_shots)))
        print("E0_meas: " + str(sum(energy_shots) / len(energy_shots)))
        print("|E0 - E0_meas|: " + str(abs(ground_state_energy - sum(energy_shots) / len(energy_shots))))        


starting matlab engine..
Degree: 2	 Trotter steps: 1
mean overlap: 0.004402737383617624
min overlap: 8.688429709278582e-38
max overlap: 0.22970645928354058
E0_meas: -0.6250026711355263
|E0 - E0_meas|: 22.01898915569125

starting matlab engine..

stopping matlab engine..
Degree: 4	 Trotter steps: 1
mean overlap: 0.0021432204792445285
min overlap: 5.4626880888199265e-39
max overlap: 0.052937711354602
E0_meas: -0.4294673624449399
|E0 - E0_meas|: 22.21452446438184

starting matlab engine..

stopping matlab engine..
Degree: 6	 Trotter steps: 1
mean overlap: 0.0021578755192159372
min overlap: 4.564369398402155e-38
max overlap: 0.09665849148178304
E0_meas: -0.25082165887112023
|E0 - E0_meas|: 22.393170167955656

starting matlab engine..

stopping matlab engine..
Degree: 8	 Trotter steps: 1
mean overlap: 0.0022320989158883683
min overlap: 2.215575988507853e-37
max overlap: 0.07601854690995194
E0_meas: -0.24043542625536707
|E0 - E0_meas|: 22.40355640057141

starting matlab engine..

stopping m